![Shopping trolley in front of a laptop](./iStock-1249219777.jpg)

It's simple to buy any product with a click and have it delivered to your door. Online shopping has been rapidly evolving over the last few years, making our lives easier. But behind the scenes, e-commerce companies face a complex challenge that needs to be addressed. 

Uncertainty plays a big role in how the supply chains plan and organize their operations to ensure that the products are delivered on time. These uncertainties can lead to challenges such as stockouts, delayed deliveries, and increased operational costs.

You work for the Sales & Operations Planning (S&OP) team at a multinational e-commerce company. They need your help to assist in planning for the upcoming end-of-the-year sales. They want to use your insights to plan for promotional opportunities and manage their inventory. This effort is to ensure they have the right products in stock when needed and ensure their customers are satisfied with the prompt delivery to their doorstep.


## The Data

You are provided with a sales dataset to use. A summary and preview are provided below.

# Online Retail.csv

| Column     | Description              |
|------------|--------------------------|
| `'InvoiceNo'` | A 6-digit number uniquely assigned to each transaction |
| `'StockCode'` | A 5-digit number uniquely assigned to each distinct product |
| `'Description'` | The product name |
| `'Quantity'` | The quantity of each product (item) per transaction |
| `'UnitPrice'` | Product price per unit |
| `'CustomerID'` | A 5-digit number uniquely assigned to each customer |
| `'Country'` | The name of the country where each customer resides |
| `'InvoiceDate'` | The day and time when each transaction was generated `"MM/DD/YYYY"` |
| `'Year'` | The year when each transaction was generated |
| `'Month'` | The month when each transaction was generated |
| `'Week'` | The week when each transaction was generated (`1`-`52`) |
| `'Day'` | The day of the month when each transaction was generated (`1`-`31`) |
| `'DayOfWeek'` | The day of the weeke when each transaction was generated <br>(`0` = Monday, `6` = Sunday) |

In [15]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.sql.functions import col, dayofmonth, month, year,  to_date, to_timestamp, weekofyear, dayofweek
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator

Analyze the `Online Retail.csv` dataset and build a forecasting model to predict `'Quantity'` of products sold.

- Split the data into two sets based on the splitting date, `"2011-09-25"`. All data up to and including this date should be in the training set, while data after this date should be in the test set. Return a pandas DataFrame, `pd_daily_train_data`, containing, at least, the columns `"Country"`, `"StockCode"`, `"InvoiceDate"`, `"Quantity"`.

In [16]:
my_spark = SparkSession.builder.appName("SalesForecast").getOrCreate()

# Importing sales data
sales_data = my_spark.read.csv("Online Retail.csv", header=True, inferSchema=True, sep=",")

# Convert InvoiceDate to datetime 
sales_data = sales_data.withColumn("InvoiceDate", to_date(to_timestamp(col("InvoiceDate"), "d/M/yyyy H:mm")))

# Aggregate data into daily intervals
daily_sales_data = sales_data.groupBy("Country", "StockCode", "InvoiceDate", "Year", "Month", "Day", "Week", "DayOfWeek").agg({"Quantity": "sum",                                                                                                           "UnitPrice": "avg"})

daily_sales_data = daily_sales_data.withColumnRenamed("sum(Quantity)", "Quantity")

In [17]:
split_date_train_test = "2011-09-25"

# Creating the train and test datasets
train_data = daily_sales_data.filter(col("InvoiceDate") <= split_date_train_test)
test_data = daily_sales_data.filter(col("InvoiceDate") > split_date_train_test)

pd_daily_train_data = train_data.toPandas()
print(pd_daily_train_data)

               Country StockCode  ... avg(UnitPrice)  Quantity
0       United Kingdom     22912  ...           4.95         3
1               France     22659  ...           1.95        24
2       United Kingdom     21544  ...           0.85        12
3       United Kingdom     21098  ...           1.25        16
4               Norway     85150  ...           2.55        12
...                ...       ...  ...            ...       ...
175447  United Kingdom     23240  ...           4.15         6
175448  United Kingdom     21908  ...           2.10         3
175449  United Kingdom     23580  ...           1.95         9
175450  United Kingdom    90200D  ...           4.25        12
175451  United Kingdom     21213  ...           0.55        48

[175452 rows x 10 columns]


- Using your test set, calculate the Mean Absolute Error (MAE) for your forecast model for the `'Quantity'` sold? Return a double (float) named mae.

In [18]:
country_indexer = StringIndexer(inputCol="Country", outputCol="CountryIndex").setHandleInvalid("keep")
stock_code_indexer = StringIndexer(inputCol="StockCode", outputCol="StockCodeIndex").setHandleInvalid("keep")


feature_cols = ["CountryIndex", "StockCodeIndex", "Month", "Year", "DayOfWeek", "Day", "Week"]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

rf = RandomForestRegressor(
    featuresCol="features",
    labelCol="Quantity",
    maxBins=4000
)

pipeline = Pipeline(stages=[country_indexer, stock_code_indexer, assembler, rf])

# Training the model
model = pipeline.fit(train_data)

# Getting test predictions
test_predictions = model.transform(test_data)
test_predictions = test_predictions.withColumn("prediction", col("prediction").cast("double"))

mae_evaluator = RegressionEvaluator(
    labelCol="Quantity", predictionCol="prediction", metricName="mae")

mae = mae_evaluator.evaluate(test_predictions)
print(mae)

25/05/14 07:31:14 WARN DAGScheduler: Broadcasting large task binary with size 1089.4 KiB
25/05/14 07:31:15 WARN DAGScheduler: Broadcasting large task binary with size 1627.2 KiB


25/05/14 07:31:16 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB


9.472060129192593


- How many units are expected to be sold during the week `39` of 2011? Store as an integer variable called `quantity_sold_w39`.

In [19]:
weekly_test_predictions = test_predictions.groupBy("Year", "Week").agg({"prediction": "sum"})

promotion_week = weekly_test_predictions.filter((col('Week')==39) & (col('Year')==2011))

quantity_sold_w39 = int(promotion_week.select("sum(prediction)").collect()[0][0])

my_spark.stop()